<a href="https://colab.research.google.com/github/nishantgovil03/Research/blob/master/Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%pwd


'/content'

In [6]:
%ls 

drive/  sample_data/


In [7]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [8]:
import pandas as pd

In [9]:
data = pd.read_csv("InputData.csv")

In [10]:
data

,Unnamed: 0,Articles without stop words,Article with stop words,Summary
0,0,budget set scene electiongordon brown seek put...,budget to set scene for electiongordon brown w...,<START> increase in the stamp duty threshold f...
1,1,army chiefs regiments decisionmilitary chiefs ...,army chiefs in regiments decisionmilitary chie...,<START> they are very much not for the good an...
2,2,howard denies split id cardsmichael howard den...,howard denies split over id cardsmichael howar...,<START> michael howard has denied his shadow c...
3,3,observers monitor uk electionministers invite ...,observers to monitor uk electionministers will...,<START> the report said individual registratio...
4,4,kilroy names election seat show host robert co...,kilroy names election seat show host robert is...,<START> ukip roger has said he is glad to see ...
...,...,...,...,...
4445,4445,india opens skies competitionindia allow domes...,india opens skies to competitionindia will all...,<START> india will allow domestic commercial a...
4446,4446,yukos bankruptcy us authorities abide us court...,yukos bankruptcy us authorities do not have to...,<START> yukos says a us court was entitled to ...
4447,4447,survey confirms property slowdowngovernment fi...,survey confirms property slowdowngovernment fi...,<START> house prices were higher on the year i...
4448,4448,high fuel prices hit ba profitsbritish airways...,high fuel prices hit ba profitsbritish airways...,<START> rod ba chief said the results were res...


In [12]:
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,LSTM,Bidirectional,Flatten,Dropout,BatchNormalization,Embedding,Input,TimeDistributed
from sklearn.model_selection import train_test_split

In [13]:
X_train,X_val,Y_train,Y_val=train_test_split(data['Article with stop words'],data['Summary'],test_size=0.3,random_state=29)
print(len(X_train),len(Y_train))
print(len(X_val),len(Y_val))

3115 3115
1335 1335


In [14]:
max_art_len=500
max_sum_len=100

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences